In [1]:
import json, requests
import pandas as pd
import time

# Foursquare

In [ ]:
# api settings
client_id = os.environ['']
client_secret = os.environ['']
v = '20180323'

# getting restaurants in the neighborhood (NYC)
location = '40.7243,-74.0018'
url = 'https://api.foursquare.com/v2/venues/explore'

params = {
    'client_id': client_id,
    'client_secret': client_secret,
    'v': v,
    'll': location,
    'query': 'restaurant',
    'limit': 50,
    'radius': 1000
}

resp = requests.get(url=url, params=params)
data = resp.json()

In [ ]:
# restaurant details (rating,name,place, ...)

all_restaurants = []
for restaurant in data['response']['groups'][0]['items']:
    id_restaurant = restaurant['venue']['id']
    
    url = f'https://api.foursquare.com/v2/venues/{id_restaurant}'
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'v': v
    }
    
    resp = requests.get(url=url, params=params)
    
    if resp.status_code == 200:
        restaurant_data = resp.json()
        venue = restaurant_data['response']['venue']

        all_restaurants.append(
            {
            'name': venue['name'],
            'address': ', '.join(venue['location']['formattedAddress']),
            'url': venue['url'],
            'price_category': venue['price']['tier'] if venue.get('price') else None,
            'likes': venue['likes']['count'],
            'rating': venue['rating'],
            'api': 'foursquare',
        })

In [ ]:
df_fs = pd.DataFrame(all_restaurants)

In [ ]:
df_fs

# Yelp

In [ ]:
# api settings
apikey = os.environ['']

headers = {
        'Authorization': f'Bearer {apikey}'
    }

# # getting restaurants in the neighborhood (NYC)
url_params = {
        'term': term.replace(' ', '+'),
        'latitude': '40.7243',
        'longitude': '-74.0018',
        'term': 'restaurants',
        'limit': 50,
        'radius': 1000
    }

url = 'https://api.yelp.com/v3/businesses/search'

resp = requests.get(url=url, headers=headers, params=url_params)
data = resp.json()

In [ ]:
# # restaurant details (rating,name,place, ...)

all_restaurants = []
for restaurant in data['businesses']:
    id_restaurant = restaurant['id']
    
    url = f'https://api.yelp.com/v3/businesses/{id_restaurant}'
    resp = requests.get(url=url, headers=headers)
    
    if resp.status_code == 200:
        restaurant_data = resp.json()
        
        all_restaurants.append(
            {
            'name': restaurant_data['name'],
            'address': ', '.join(restaurant_data['location']['display_address']),
            'url': restaurant_data['url'],
            'price_category': restaurant_data['price'].strip() if restaurant_data.get('price') else None,
            'likes': None,
            'rating': restaurant_data['rating']*2,
            'api': 'yelp',
        })

In [ ]:
df_yelp = pd.DataFrame(all_restaurants)
df_yelp['price_category'] = df_yelp['price_category'].apply(lambda x: len(x.strip()) if isinstance(x,str) else None)

In [ ]:
df_yelp

# Google 

In [17]:
# key = os.environ['']
key = os.environ['']
radius = 1000
query = 'restaurant'
location = '40.7243,-74.0018'

In [18]:
resp = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={location}&radius={radius}&type={query}&key={key}')
data = resp.json()

results = []
results += data['results']
next_page_token = data.get('next_page_token')

while next_page_token:
    resp = requests.get(f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next_page_token}&key={key}')
    data = resp.json()
    
    results += data['results']
    next_page_token = data.get('next_page_token')

In [19]:
all_restaurants = []
for restaurant in results:
    all_restaurants.append(
            {
            'name': restaurant['name'],
            'address': restaurant['vicinity'],
            'url': None,
            'price_category': restaurant['price_level'],
            'likes': None,
            'rating': restaurant['rating']*2,
            'api': 'google',
        })
    

In [20]:
df_google = pd.DataFrame(all_restaurants)

In [22]:
# df_google

# Database

In [ ]:
import sqlite3

In [ ]:
# create connection
conn = sqlite3.connect("restaurants.db")

In [ ]:
# insert into db
df_fs.to_sql("restaurant_info", conn, if_exists="append", index=False)
df_yelp.to_sql("restaurant_info", conn, if_exists="append", index=False)
df_google.to_sql("restaurant_info", conn, if_exists="append", index=False)

In [ ]:
# all data from db
df_all = pd.read_sql('Select * from restaurant_info', conn)

In [ ]:
# top 10 restaurants according to rating
df_all_top = df_all.sort_values('rating',ascending=False).head(10)

In [24]:
df_all_top

,name,address,url,price_category,likes,rating,api
0,NOMO SOHO,"9 Crosby Street, New York",None,3,None,8.4,google
1,Artichoke Basille's Pizza,"111 MacDougal Street, New York",None,2,None,8.8,google
2,Gilligan's,"310 West Broadway, New York",None,3,None,8.2,google
3,Lombardi's,"32 Spring Street, New York",None,2,None,8.2,google
4,City Winery,"155 Varick Street, New York",None,3,None,8.6,google
5,Cipriani Downtown,"376 West Broadway, New York",None,4,None,8.4,google
6,Carbone,"181 Thompson Street, New York",None,3,None,8.8,google
7,Boqueria,"171 Spring Street, New York",None,2,None,8.8,google
8,Macao Trading Co,"311 Church Street, New York",None,3,None,9.0,google
9,La Esquina - SoHo,"114 Kenmare Street, New York",None,2,None,8.4,google


## Google Directions

In [42]:
mode = "driving"

In [43]:
df_all_top["key"] = 0

In [44]:
direction_matrix = df_all_top[["address","key"]].merge(df_all_top[["address","key"]], on="key")[["address_x","address_y"]]

In [60]:
duration = []
for row in direction_matrix.iterrows():
    row = row[1]
    origin = row["address_x"].replace(",","").replace(" ","+")
    destination = row["address_y"].replace(",","").replace(" ","+")
    resp = requests.get(f'https://maps.googleapis.com/maps/api/directions/json?origin={origin}&destination={destination}&key={key}&mode={mode}')
    data = resp.json()
    duration.append(data["routes"][0]["legs"][0]["duration"]["value"])
    

In [63]:
direction_matrix["duration"] = duration

In [77]:
direction_matrix_final = direction_matrix.set_index(["address_y","address_x"]).unstack()

## Travelling Salesman Problem

This section simply follow steps from the [ortools tutorial](https://developers.google.com/optimization/routing/tsp).

In [82]:
from ortools.constraint_solver import pywrapcp

In [79]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = direction_matrix_final.values
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data

In [80]:
df = create_data_model()

In [83]:
manager = pywrapcp.RoutingIndexManager(len(df['distance_matrix']),
                                       df['num_vehicles'], df['depot'])
routing = pywrapcp.RoutingModel(manager)

In [84]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return df['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [85]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [89]:
from ortools.constraint_solver import routing_enums_pb2

In [90]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [91]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {} miles'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Route distance: {}miles\n'.format(route_distance)

In [93]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 3233 miles
Route for vehicle 0:
 0 -> 17 -> 6 -> 16 -> 9 -> 7 -> 14 -> 5 -> 19 -> 3 -> 11 -> 8 -> 1 -> 18 -> 13 -> 2 -> 15 -> 12 -> 4 -> 10 -> 0

